In [ ]:
## Tools
## Wikipedia and arxiv
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper  

In [ ]:
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results = 1 )